<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


# TO SEE MAPs (AND ENJOY USING THEM) CLICK ABOVE LINK TO OPEN IN GOOGLE COLAB

Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
!pip3 install folium
!pip3 install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=f5238a6b6ed53de161ff987bf87c368b220f52f763af7ecfe35c6e38937bc4b2
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [ ]:
import folium
import wget
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [ ]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
# grouped by launch site = get lat and long of each site
# .first() set Launch Site column as 1st column
# as_index = False, alternative to .reset_index()
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
# launch_site df remove 'class' column
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a red circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a red circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a red yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [ ]:
# launch site locations
CCAFSLC40_coordinate = [28.562302,-80.577356]
CCAFSSLC40_coordinate = [28.563197,-80.576820]
KSCLC39A_coordinate = [28.573255,-80.646895]
VAFBSLC4E_coordinate = [34.632834,-120.610745]

In [ ]:
# launch site red circles 
# popup label showing name
CCAFSLC40_circle = folium.Circle(CCAFSLC40_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Cape Canaveral Launch Complex 40'))
CCAFSSLC40_circle = folium.Circle(CCAFSSLC40_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Cape Canaveral Space Launch Complex 40'))
KSCLC39A_circle = folium.Circle(KSCLC39A_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Kennedy Space Center Launch Complex 39A'))
VAFBSLC4E_circle = folium.Circle(VAFBSLC4E_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Vandenberg AFB Space Launch Complex 4'))

In [ ]:
# launch site icons with name
CCAFSLC40_marker = folium.map.Marker(
    CCAFSLC40_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',
        )
    )

CCAFSSLC40_marker = folium.map.Marker(
    CCAFSSLC40_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',
        )
    )

KSCLC39A_marker = folium.map.Marker(
    KSCLC39A_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A',
        )
    )

VAFBSLC4E_marker = folium.map.Marker(
    VAFBSLC4E_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
        )
    )

In [ ]:
# add circles, popups and labels to the map site_map
site_map.add_child(CCAFSLC40_circle) 
site_map.add_child(CCAFSSLC40_circle) 
site_map.add_child(KSCLC39A_circle) 
site_map.add_child(VAFBSLC4E_circle)
site_map.add_child(CCAFSLC40_marker)
site_map.add_child(CCAFSSLC40_marker)
site_map.add_child(KSCLC39A_marker)
site_map.add_child(VAFBSLC4E_marker)

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
Yes
*   Are all launch sites in very close proximity to the coast?
Yes

Explain More:

* Equator proximity: 
Maximum rotational speed of lies on the Equator (average equatorial speed: 1,040 mph or 1,674 kmph); this saves reduces launch cost by decreasing need for extra fuel and boosters.
* Coast proximity: 
In case of launch failure, coastal proximity mitigates risk from burning debris.  
* Eastern coast launch sites:
The rotational direction is west to east, thus launching eastward uses rotational speed and direction to reduce launch cost.  An eastward traveling launch failure would be falling into the ocean.


In [ ]:
# the Equator is located at 0 degrees Latitude
# what are the distances of the launch sites from the equator
!pip install haversine
import haversine as hs
equator1 = [0,-80.577356]
equator2 = [0,-80.576820]
equator3 = [0,-80.646895]
equator4 = [0,-120.610745]
# launch site locations
CCAFSLC40_coordinate = [28.562302,-80.577356]
CCAFSSLC40_coordinate = [28.563197,-80.576820]
KSCLC39A_coordinate = [28.573255,-80.646895]
VAFBSLC4E_coordinate = [34.632834,-120.610745]
# calculate Haversine Distance to the equator 
print(hs.haversine(equator1,CCAFSLC40_coordinate), 'km for CCAFS LC-40')
print(hs.haversine(equator2,CCAFSSLC40_coordinate), 'km for CCAFS SLC-40')
print(hs.haversine(equator3,KSCLC39A_coordinate), 'km for KSC LC-39A')
print(hs.haversine(equator4,VAFBSLC4E_coordinate), 'km for VAFBS LC-4E')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
3175.987462544397 km for CCAFS LC-40
3176.086982141206 km for CCAFS SLC-40
3177.205382258195 km for KSC LC-39A
3851.000755344627 km for VAFBS LC-4E


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
# 0 = failure - 1 = success
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [ ]:
# create a MarkerCluster object
marker_cluster = MarkerCluster()

In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

In [ ]:
# Create a new column in launch_sites dataframe called marker_color to store the marker colors based on the class value
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [ ]:
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker([record['Lat'], record['Long']], 
                  icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
# California
CA1_hwy = [34.63005, -120.43044]
CA_coastline = [34.62779, -120.62635]
# Florida
US1_hwy = [28.57135, -80.79917]
FL_coastline = [28.57075, -80.57011]

In [ ]:
# INSTEAD of the below, I will use the Haversine Distance method
# import haversine as hs

# launch site locations
CCAFSLC40_coordinate = [28.562302,-80.577356]
CCAFSSLC40_coordinate = [28.563197,-80.576820]
KSCLC39A_coordinate = [28.573255,-80.646895]
VAFBSLC4E_coordinate = [34.632834,-120.610745]

# calculate Haversine Distance to the highways and coastlines 

# Florida
print(hs.haversine(US1_hwy,CCAFSLC40_coordinate), 'km for CCAFS LC-40 to US1 Highway')
print(hs.haversine(US1_hwy,CCAFSSLC40_coordinate), 'km for CCAFS SLC-40 to US1 Highway')
print(hs.haversine(US1_hwy,KSCLC39A_coordinate), 'km for KSC LC-39A to US1 Highway')
print(hs.haversine(FL_coastline,CCAFSLC40_coordinate), 'km for CCAFS LC-40 to FL Coastline')
print(hs.haversine(FL_coastline,CCAFSSLC40_coordinate), 'km for CCAFS SLC-40 to FL Coastline')
print(hs.haversine(FL_coastline,KSCLC39A_coordinate), 'km for KSC LC-39A to FL Coastline')

# California
print(hs.haversine(CA1_hwy,VAFBSLC4E_coordinate), 'km for VAFBS LC-4E to CA1 Highway')
print(hs.haversine(CA_coastline,VAFBSLC4E_coordinate), 'km for VAFBS LC-4E to CA Coastline')

21.685302098907815 km for CCAFS LC-40 to US1 Highway
21.733119632517038 km for CCAFS SLC-40 to US1 Highway
14.871634560954854 km for KSC LC-39A to US1 Highway
1.1760834031582565 km for CCAFS LC-40 to FL Coastline
1.065250192975602 km for CCAFS SLC-40 to FL Coastline
7.503486902133204 km for KSC LC-39A to FL Coastline
16.499736934492823 km for VAFBS LC-4E to CA1 Highway
1.5339949017261247 km for VAFBS LC-4E to CA Coastline


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [ ]:
#from math import sin, cos, sqrt, atan2, radians

#def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    #R = 6373.0

    #lat1 = radians(lat1)
    #lon1 = radians(lon1)
    #lat2 = radians(lat2)
    #lon2 = radians(lon2)

    #dlon = lon2 - lon1
    #dlat = lat2 - lat1

    #a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    #c = 2 * atan2(sqrt(a), sqrt(1 - a))

    #distance = R * c
    #return distance
    
# find coordinate of the closet coastline e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
# distance
# create a folium.Marker to show the distance

CCAFSLC40andCCAFSSLC40toUS1HWYdistance = 21.685302098907815
KSCLC39AtoUS1HWYdistance = 14.871634560954854 
CCAFSLC40andCCAFSSLC40toFLCoastlinedistance = 1.065250192975602
KSCLC39AtoFLCoastlinedistance = 7.503486902133204
VAFBSLC4EtoCA1HWYdistance = 16.499736934492823
VAFBSLC4EtoCACoastlinedistance = 1.5339949017261247

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(distance),
#        )
#    )

In [ ]:
# California 

VAFBSLC4E_coordinate = [34.632834,-120.610745]

CA1_hwy = [34.63005, -120.43044]
CA_coastline = [34.62779, -120.62635]

# Florida

CCAFSLC40_coordinate = [28.562302,-80.577356]
CCAFSSLC40_coordinate = [28.563197,-80.576820]
KSCLC39A_coordinate = [28.573255,-80.646895]

US1_hwy = [28.57135, -80.79917]
FL_coastline = [28.57075, -80.57011]

In [ ]:
# Create and add a folium.Marker for closest coastlines and highways point on the map
# Display the distance between these points and launch sites using the icon property 

CCAFSLC40andCCAFSSLC40toUS1HWYdistance_marker = folium.Marker(
    US1_hwy,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(CCAFSLC40andCCAFSSLC40toUS1HWYdistance),
        )
    )

KSCLC39AtoUS1HWYdistance_marker = folium.Marker(
    US1_hwy,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(KSCLC39AtoUS1HWYdistance),
        )
    )

CCAFSLC40andCCAFSSLC40toFLCoastlinedistance_marker = folium.Marker(
    FL_coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(CCAFSLC40andCCAFSSLC40toFLCoastlinedistance),
        )
    )

KSCLC39AtoFLCoastlinedistance_marker = folium.Marker(
    FL_coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(KSCLC39AtoFLCoastlinedistance),
        )
    )

VAFBSLC4EtoCA1HWYdistance_marker = folium.Marker(
    CA1_hwy,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(VAFBSLC4EtoCA1HWYdistance),
        )
    )

VAFBSLC4EtoCACoastlinedistance_marker = folium.Marker(
    CA_coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(VAFBSLC4EtoCACoastlinedistance),
        )
    )

In [ ]:
# add to the map

site_map.add_child(CCAFSLC40andCCAFSSLC40toUS1HWYdistance_marker) 
site_map.add_child(KSCLC39AtoUS1HWYdistance_marker) 
site_map.add_child(CCAFSLC40andCCAFSSLC40toFLCoastlinedistance_marker) 
site_map.add_child(KSCLC39AtoFLCoastlinedistance_marker)
site_map.add_child(VAFBSLC4EtoCA1HWYdistance_marker)
site_map.add_child(VAFBSLC4EtoCACoastlinedistance_marker)

site_map

In [ ]:
# Create a `folium.PolyLine` object using the coastline/highway coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates1 = [VAFBSLC4E_coordinate, CA1_hwy]
coordinates2 = [VAFBSLC4E_coordinate, CA_coastline]
coordinates3 = [CCAFSLC40_coordinate, US1_hwy]
coordinates4 = [CCAFSSLC40_coordinate, US1_hwy]
coordinates5 = [KSCLC39A_coordinate, US1_hwy]
coordinates6 = [CCAFSLC40_coordinate, FL_coastline]
coordinates7 = [CCAFSSLC40_coordinate, FL_coastline]
coordinates8 = [KSCLC39A_coordinate, FL_coastline]

lines1=folium.PolyLine(locations=coordinates1, weight=1)
lines2=folium.PolyLine(locations=coordinates2, weight=1)
lines3=folium.PolyLine(locations=coordinates3, weight=1)
lines4=folium.PolyLine(locations=coordinates4, weight=1)
lines5=folium.PolyLine(locations=coordinates5, weight=1)
lines6=folium.PolyLine(locations=coordinates6, weight=1)
lines7=folium.PolyLine(locations=coordinates7, weight=1)
lines8=folium.PolyLine(locations=coordinates8, weight=1)

site_map.add_child(lines1)
site_map.add_child(lines2)
site_map.add_child(lines3)
site_map.add_child(lines4)
site_map.add_child(lines5)
site_map.add_child(lines6)
site_map.add_child(lines7)
site_map.add_child(lines8)

Draw a line between a launch site to its closest city/railway. Use `MousePosition` to find the their coordinates on the map first


In [ ]:
# Create a marker with distance to a closest city, railway
# Draw a line between the marker to the launch site

In [ ]:
#Florida
FL_rail = [28.5707, -80.80401]
FL_melbourne = [28.09402, -80.64395]

CCAFSLC40_coordinate = [28.562302,-80.577356]
CCAFSSLC40_coordinate = [28.563197,-80.576820]
KSCLC39A_coordinate = [28.573255,-80.646895]

#California 
VAFBSLC4E_coordinate = [34.632834,-120.610745]

CA_rail = [34.63542, -120.45778]
CA_lompoc = [34.6389, -120.45778]

In [ ]:
# calculate Haversine Distance 
print("FLrailtoCCAFSLC40distance =", hs.haversine(FL_rail,CCAFSSLC40_coordinate))
print("FLrailtoKSCLC39Adistance =",hs.haversine(FL_rail,KSCLC39A_coordinate))
      
print("FLmelbournetoCCAFSLC40distance =", hs.haversine(FL_melbourne,CCAFSSLC40_coordinate))
print("FLmelbournetoKSCLC39Adistance =", hs.haversine(FL_melbourne,KSCLC39A_coordinate))
      
print("CArailtoVAFBSLC4Edistance =", hs.haversine(CA_rail,VAFBSLC4E_coordinate))
print("CAlompoctoVAFBSLC4Edistance =", hs.haversine(CA_lompoc,VAFBSLC4E_coordinate))

FLrailtoCCAFSLC40distance = 22.202615498003198
FLrailtoKSCLC39Adistance = 15.345444375524757
FLmelbournetoCCAFSLC40distance = 52.582308004438616
FLmelbournetoKSCLC39Adistance = 53.28935380304477
CArailtoVAFBSLC4Edistance = 13.997886958758214
CAlompoctoVAFBSLC4Edistance = 14.01088503742087


In [ ]:
# distances
FLrailtoCCAFSLC40distance = 22.202615498003198
FLrailtoKSCLC39Adistance = 15.345444375524757
FLmelbournetoCCAFSLC40distance = 52.582308004438616
FLmelbournetoKSCLC39Adistance = 53.28935380304477
CArailtoVAFBSLC4Edistance = 13.997886958758214
CAlompoctoVAFBSLC4Edistance = 14.01088503742087

In [ ]:
FLrailtoCCAFSLC40distance_marker = folium.Marker(
    FL_rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(FLrailtoCCAFSLC40distance),
        )
    )

FLrailtoKSCLC39Adistance_marker = folium.Marker(
    FL_rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(FLrailtoKSCLC39Adistance),
        )
    )

FLmelbournetoCCAFSLC40distance_marker = folium.Marker(
    FL_melbourne,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(FLmelbournetoCCAFSLC40distance),
        )
    )

FLmelbournetoKSCLC39Adistance_marker = folium.Marker(
    FL_melbourne,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(FLmelbournetoKSCLC39Adistance),
        )
    )

CArailtoVAFBSLC4Edistance_marker = folium.Marker(
    CA_rail,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(CArailtoVAFBSLC4Edistance),
        )
    )

CAlompoctoVAFBSLC4Edistance_marker = folium.Marker(
    CA_lompoc,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{} KM".format(CAlompoctoVAFBSLC4Edistance),
        )
    )


In [ ]:
# Create a `folium.PolyLine` object using the rail/city coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates1 = [VAFBSLC4E_coordinate, CA_rail]
coordinates2 = [VAFBSLC4E_coordinate, CA_lompoc]
coordinates3 = [CCAFSLC40_coordinate, FL_rail]
coordinates4 = [CCAFSSLC40_coordinate, FL_rail]
coordinates5 = [KSCLC39A_coordinate, FL_rail]
coordinates6 = [CCAFSLC40_coordinate, FL_melbourne]
coordinates7 = [CCAFSSLC40_coordinate, FL_melbourne]
coordinates8 = [KSCLC39A_coordinate, FL_melbourne]
coordinates9 = [VAFBSLC4E_coordinate, CA_rail]
coordinates10 = [VAFBSLC4E_coordinate, CA_lompoc]

lines1=folium.PolyLine(locations=coordinates1, weight=1)
lines2=folium.PolyLine(locations=coordinates2, weight=1)
lines3=folium.PolyLine(locations=coordinates3, weight=1)
lines4=folium.PolyLine(locations=coordinates4, weight=1)
lines5=folium.PolyLine(locations=coordinates5, weight=1)
lines6=folium.PolyLine(locations=coordinates6, weight=1)
lines7=folium.PolyLine(locations=coordinates7, weight=1)
lines8=folium.PolyLine(locations=coordinates8, weight=1)
lines9=folium.PolyLine(locations=coordinates9, weight=1)
lines10=folium.PolyLine(locations=coordinates10, weight=1)

In [ ]:
site_map.add_child(FLrailtoCCAFSLC40distance_marker) 
site_map.add_child(FLrailtoKSCLC39Adistance_marker) 
site_map.add_child(FLmelbournetoCCAFSLC40distance_marker) 
site_map.add_child(FLmelbournetoKSCLC39Adistance_marker) 
site_map.add_child(CArailtoVAFBSLC4Edistance_marker) 
site_map.add_child(CAlompoctoVAFBSLC4Edistance_marker) 

site_map.add_child(lines1)
site_map.add_child(lines2)
site_map.add_child(lines3)
site_map.add_child(lines4)
site_map.add_child(lines5)
site_map.add_child(lines6)
site_map.add_child(lines7)
site_map.add_child(lines8)
site_map.add_child(lines9)
site_map.add_child(lines10)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
Yes
*   Are launch sites in close proximity to highways?
Yes
*   Are launch sites in close proximity to coastline?
Yes
*   Do launch sites keep certain distance away from cities?
Yes

Explain More:
* Railways: launch sites are located to the east of railways, the risk of launch failure damage is low
* Highways: further distance is allocated between highways and launch sites, Florida launch sites are east of highways, in California closest proximity is north of the launch site with a wide berth to the east
* Coastline: launch sites are close to coastlines, for reasons as mentioned previously, additionally this guarantees a non-populous adjacent region
* Cities: launch sites in Florida have Melbourne ~50 km south, the launch site in California has a small city ~ 15 km to the east, and Santa Barbara more than 5 times further away; major cities of Orlando and Los Angelas are given an even wider berth 


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
